In [ ]:
import win32com.client
import win32com
import os
import sys

In [ ]:
#outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
outlook = win32com.client.Dispatch("Outlook.Application").Session.Stores
accounts= win32com.client.Dispatch("Outlook.Application").Session.Accounts;

In [ ]:
for account in outlook:
     print(account.DisplayName)

In [ ]:
def emailleri_al(folder):
    messages = folder.Items
    a=len(messages)
    if a>0:
        for message2 in messages:
             try:
                sender = message2.Subject
                if sender != "":
                    print(sender)
             except:
                print("Error")
                print(account.DeliveryStore.DisplayName)
                pass

             try:
                message2.Save
                message2.Close(0)
             except:
                 pass


In [ ]:
for account in accounts:
    global inbox
    inbox = outlook.Folders(account.DeliveryStore.DisplayName)
    print("****Account Name**********************************")
    print(account.DisplayName)
    print("***************************************************")
    folders = inbox.Folders

    for folder in folders:
        print("****Folder Name**********************************")
        print(folder)
        print("*************************************************")
#         emailleri_al(folder)
        a = len(folder.folders)

        if a>0 :
            global z
            z = outlook.Folders(account.DeliveryStore.DisplayName).Folders(folder.name)
            x = z.Folders
            for y in x:
                if y.name == 'batch':
                    emailleri_al(y)
                    print("****Folder Name**********************************")
                    print("..."+y.name)
                    print("*************************************************")



In [ ]:
import win32com.client
import os
outlook=win32com.client.Dispatch("Outlook.Application").GetNameSpace("MAPI")
inbox=outlook.Folders("SD_SD-Drift") #Inbox default index value is 6
message=inbox.Items
message2=message.GetLast()
# subject=message2.Subject
body=message2.body
date=message2.senton.date()   
# sender=message2.Sender
# attachments=message2.Attachments
# print(subject)
# print(body)
# print(sender)
# print(attachments.count)
print(date)

In [ ]:
import win32com.client

for stor in win32com.client.Dispatch("Outlook.Application").Session.Stores:
    print( stor.DisplayName) 

In [ ]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
folder = outlook.Folders("SD_SD-Drift")
inbox = folder.Folders("Inbox")
msg = inbox.Items

In [ ]:
outlook=win32com.client.Dispatch("Outlook.Application").GetNameSpace("MAPI")


your_folder = outlook.Folders["SD_SD-Drift"].Folders['Inbox'].Folders['AftenVagt']
for message in your_folder.Items:
    print(message.Subject)

In [ ]:
import win32com.client

search_string = 'Selvaraj'

outlook = win32com.client.gencache.EnsureDispatch('Outlook.Application')
gal = outlook.Session.GetGlobalAddressList()
entries = gal.AddressEntries
ae = entries[search_string]
email_address = None
if 'EX' == ae.Type:
    eu = ae.GetExchangeUser()
    email_address = eu.PrimarySmtpAddress
    
if 'SMTP' == ae.Type:
    email_address = ae.Address

print('Email address: ', email_address)

# https://docs.microsoft.com/en-us/office/vba/api/outlook.exchangeuser

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

nameAliasDict = {}

for i in contacts:
    name = i.Name
    alias = i.Address.split("=")[-1]
    nameAliasDict[alias] = name

print( "\nThe global address list contains",numEntries,"entries.")

In [ ]:
import win32com.client

DEBUG = 0

class MSOutlook:
    def __init__(self):
        self.outlookFound = 0
        try:
            self.oOutlookApp = \
                win32com.client.gencache.EnsureDispatch("Outlook.Application")
            self.outlookFound = 1
        except:
            print ("MSOutlook: unable to load Outlook")
        
        self.records = []


    def loadContacts(self, keys=None):
        if not self.outlookFound:
            return

        # this should use more try/except blocks or nested blocks
        onMAPI = self.oOutlookApp.GetNamespace("MAPI")
        ofContacts = \
            onMAPI.GetDefaultFolder(win32com.client.constants.olFolderContacts)

        if DEBUG:
            print ("number of contacts:", len(ofContacts.Items))

        for oc in range(len(ofContacts.Items)):
            contact = ofContacts.Items.Item(oc + 1)
            if contact.Class == win32com.client.constants.olContact:
                if keys is None:
                    # if we were't give a set of keys to use
                    # then build up a list of keys that we will be
                    # able to process
                    # I didn't include fields of type time, though
                    # those could probably be interpreted
                    keys = []
                    for key in contact._prop_map_get_:
                        if isinstance(getattr(contact, key), (int, str, unicode)):
                            keys.append(key)
                    if DEBUG:
                        keys.sort()
                        print ("Fields\n======================================")
                        for key in keys:
                            print (key)
                record = {}
                for key in keys:
                    record[key] = getattr(contact, key)
                if DEBUG:
                    print (oc, record['FullName'])
                self.records.append(record)


if __name__ == '__main__':
    if DEBUG:
        print ("attempting to load Outlook")
    oOutlook = MSOutlook()
    # delayed check for Outlook on win32 box
    if not oOutlook.outlookFound:
        print ("Outlook not found")
        sys.exit(1)

    fields = ['FullName',
                'CompanyName', 
                'MailingAddressStreet',
                'MailingAddressCity', 
                'MailingAddressState', 
                'MailingAddressPostalCode',
                'HomeTelephoneNumber', 
                'BusinessTelephoneNumber', 
                'MobileTelephoneNumber',
                'Email1Address',
                'Body'
                ]

    if DEBUG:
        import time
        print ("loading records...")
        startTime = time.time()
    # you can either get all of the data fields
    # or just a specific set of fields which is much faster
    #oOutlook.loadContacts()
    oOutlook.loadContacts(fields)
    if DEBUG:
        print ("loading took %f seconds" % (time.time() - startTime))
    for outtwr in oOutlook.records:
        print ("Contact: %s" % outtwr)
#     print ("Number of contacts: %d" % len(oOutlook.records))
#     print ("Contact: %s" % oOutlook.records[0]['FullName'])
#     print ("Body:\n%s" % oOutlook.records[0]['Body'])

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

nameAliasDict = {}

for i in contacts:
    name = i.Name
    alias = i.Address.split("=")[-1]
    nameAliasDict[alias] = name

print ("\nThe global address list contains",numEntries,"entries.")

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

nameAliasDict = {}

for i in contacts:
    name = i.Name
    alias = i.Address.split("=")[-1]
    nameAliasDict[alias] = name

print ("\nThe global address list contains",numEntries,"entries.")

In [ ]:
import win32com.client
import os
output_list = []
import pickle

def emailleri_al(folder):
    messages = folder.Items
    a=len(messages)
    if a>0:
        for message2 in messages:
             try:
                Sub = message2.Subject
                date1 = message2.senton.date()

                if Sub != "":
                    for word in Sub.split():
                        if word.startswith("L"):
                            if "/" in word:
                                word_sep = word.split('/')
                                for word_ser in word_sep:
                                    output_list.append(word_ser + ' - ' + str(date1))

                            elif ',' in word:
                                text_list = word.split(',')
                                base_txt = text_list[0]
                                for text1 in text_list:
                                    if len(text1)>1:
                                        output_list.append(text1 + ' - ' + str(date1))
                                    else:
                                        output_list.append(base_txt[:-1]+ text1 + ' - ' + str(date1))

                            else:
                                output_list.append(word + ' - ' + str(date1))
             except:
                print("Error")
                pass
             try:
                message2.Save
                message2.Close(0)
             except:
                 pass
    pickle.dump(set(output_list), open('Pickled_set_job.p',"wb"))

outlook=win32com.client.Dispatch("Outlook.Application").GetNameSpace("MAPI")
global inbox
inbox=outlook.Folders("SD_SD-Drift")
folders = inbox.Folders

for folder in folders:
    # print(type(folder.name))
    if folder.name == 'Inbox':
        print('Inbox reached')
        emailleri_al(folder)

    # a = len(folder.folders)
    # if a>0 :
    #     global z
    #     z = outlook.Folders("SD_SD-Drift").Folders(folder.name)
    #     x = z.Folders
    #     for y in x:
    #         if y.name == 'Inbox':
    #             print('Inbox reached')
                # emailleri_al(y)


In [ ]:
import codecs
import tempfile

import win32api
import win32com.client
constants = win32com.client.constants

session = win32com.client.Dispatch ("MAPI.Session")
session.Logon ()

filename = tempfile.mktemp (suffix=".csv")
print( "Writing to", filename)
f = codecs.open (filename, "w", "latin-1")
try:
  address_lists = session.AddressLists
  gal = address_lists.Item ("Global Address List")
  addresses = gal.AddressEntries
  for i in range (addresses.Count):
    entry = addresses.Item (i + 1)
    if entry.DisplayType == constants.CdoUser:
      name = entry.Name
      smtp = entry.Fields[0x39FE001E]
      f.write ('"%s","%s"\n' % (name, smtp))
      print (name)
finally:
  f.close ()

session.Logoff ()

In [ ]:
import win32com.client

search_string = "santhosh"

outlook = win32com.client.gencache.EnsureDispatch('Outlook.Application')
gal = outlook.Session.GetGlobalAddressList()
entries = gal.AddressEntries
ae = entries[search_string]
email_address = None

if 'EX' == ae.Type:
    eu = ae.GetExchangeUser()
    email_address = eu.PrimarySmtpAddress

if 'SMTP' == ae.Type:
    email_address = ae.Address

print('Email address: ', email_address)

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

nameAliasDict = {}

for i in contacts:
    name = i.Name
    alias = i.Address.split("=")[-1]
    nameAliasDict[alias] = name

print ("\nThe global address list contains",numEntries,"entries.")
print(nameAliasDict)

In [ ]:
import win32com.client

search_string = "santhosh"

outlook = win32com.client.gencache.EnsureDispatch('Outlook.Application')
gal = outlook.Session.GetGlobalAddressList()
entries = gal.AddressEntries
ae = entries[search_string]
email_address = None

if 'EX' == ae.Type:
    eu = ae.GetExchangeUser()
    email_address = eu.PrimarySmtpAddress

if 'SMTP' == ae.Type:
    email_address = ae.Address

print('Email address: ', email_address)

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

nameAliasDict = {}

 for i in contacts:
    name = i.Name
    alias = i.Address.split("=")[-1]
    nameAliasDict[alias] = name

print ("\nThe global address list contains",numEntries,"entries.")

In [ ]:
from __future__ import print_function
import win32com.client

search_string = 'Brian Kjær'

outlook = win32com.client.gencache.EnsureDispatch('Outlook.Application')
recipient = outlook.Session.CreateRecipient(search_string)
recipient.Resolve()
print('Resolved OK: ', recipient.Resolved)
print('Is it a sendable? (address): ', recipient.Sendable)
print('Name: ', recipient.Name)

ae = recipient.AddressEntry
email_address = None

if 'EX' == ae.Type:
    eu = ae.GetExchangeUser()
    email_address = eu.PrimarySmtpAddress
    alias = eu.Alias
if 'SMTP' == ae.Type:
    email_address = ae.Address

print('Email address: ', email_address)
print('Alias: ', alias)

In [ ]:
import win32com.client

o = win32com.client.gencache.EnsureDispatch("Outlook.Application")
ns = o.GetNamespace("MAPI")

adrLi = ns.AddressLists.Item("Global Address List")
contacts = adrLi.AddressEntries
numEntries = adrLi.AddressEntries.Count

f1 = open('contact_list.txt','w')
for i in contacts:
    name = i.Name
    print(str(name.encode("utf-8")),file=f1)
#     f1.write(name.encode("utf-8")+'\n')
f1.close()

In [ ]:
f2 = codecs.open('contact_list.txt', encoding='Latin-1')
data = f2.readlines()
for data1 in data:
    tes = data1[1:]
    print(tes.encode('utf-8'))